<a href="https://colab.research.google.com/github/DwayneLi/CFPB_Consumer_Complaints_Analysis/blob/master/Complaints_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contents of complaint analysis

## Get date and necessary packages

In [2]:
!wget -q https://www.dropbox.com/s/xzqxcw2yiz2mwie/complaints.csv

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df= pd.read_csv('complaints.csv')

## Clean data



### Merge products under same category.

There are product classifications before and after 2017-05. In order to analyze data exactlly, we put the same sub-group under new classification of products. 

First we check the duplicate sub product group and mark them by latest record date.


In [6]:
# transform date into date format
df['Date received']=df['Date received'].apply(pd.to_datetime)

In [7]:
# check the duplicate sub product group.
product=df.pivot_table(['Complaint ID','Date received'],['Product','Sub-product'],aggfunc={'Complaint ID':lambda x: x.value_counts().count(),'Date received':max})
product.reset_index(inplace=True)#.to_csv('product_list.csv')
product.head()

,Product,Sub-product,Complaint ID,Date received
0,Bank account or service,(CD) Certificate of deposit,3417,2017-04-22
1,Bank account or service,Cashing a check without an account,625,2017-04-18
2,Bank account or service,Checking account,59045,2017-04-22
3,Bank account or service,Other bank product/service,17825,2017-04-21
4,Bank account or service,Savings account,5294,2017-04-21


In [8]:
product[product['Sub-product'].duplicated(keep=False)].sort_values('Sub-product').head()

,Product,Sub-product,Complaint ID,Date received
2,Bank account or service,Checking account,59045,2017-04-22
6,Checking or savings account,Checking account,45645,2020-04-02
58,Mortgage,Conventional home mortgage,41720,2020-04-02
23,"Credit reporting, credit repair services, or o...",Conventional home mortgage,1,2018-05-22
69,Other financial service,Debt settlement,333,2017-04-21


In [12]:
# Payday loan, title loan, or personal loan
df['Product'].replace('Payday loan','Payday loan, title loan, or personal loan',inplace=True)
df.loc[df['Sub-product'] == 'Pawn loan','Product']='Payday loan, title loan, or personal loan'
df.loc[df['Sub-product'] == 'Personal line of credit','Product']='Payday loan, title loan, or personal loan'
df.loc[df['Sub-product'] == 'Installment loan','Product']='Payday loan, title loan, or personal loan'

# Money transfer, virtual currency, or money service 
df['Product'].replace('Other financial service','Money transfer, virtual currency, or money service',inplace=True)
df['Sub-product'].replace('Check cashing','Check cashing service',inplace=True)
df['Sub-product'].replace('Traveler’s/Cashier’s checks',"Traveler's check or cashier's check",inplace=True)
df['Product'].replace('Money transfers','Money transfer, virtual currency, or money service',inplace=True)
df['Product'].replace('Virtual currency','Money transfer, virtual currency, or money service',inplace=True)

# Checking or savings account
df['Product'].replace('Bank account or service','Checking or savings account',inplace=True)
df['Sub-product'].replace('(CD) Certificate of deposit',"CD (Certificate of Deposit)",inplace=True)

# Debt collection
df['Sub-product'].replace('Credit card',"Credit card debt",inplace=True)
df['Sub-product'].replace('Other (i.e. phone, health club, etc.)',"Other debt",inplace=True)
df['Sub-product'].replace('Medical',"Medical debt",inplace=True)
df['Sub-product'].replace('Payday loan',"Payday loan debt",inplace=True)
df['Sub-product'].replace('Auto',"Auto debt",inplace=True)
df['Sub-product'].replace('Mortgage',"Mortgage debt",inplace=True)
df['Sub-product'].replace('Federal student loan',"Federal student loan debt",inplace=True)
df['Sub-product'].replace('Non-federal student loan',"Private student loan debt",inplace=True)


# Credit card or prepaid card
df['Product'].replace('Prepaid card','Credit card or prepaid card',inplace=True)
df['Product'].replace('Credit card','Credit card or prepaid card',inplace=True)
df['Sub-product'].replace('Government benefit payment card',"Government benefit card",inplace=True)
df['Sub-product'].replace('ID prepaid card',"ID prepaid card",inplace=True)

# Vehicle loan or lease
df['Product'].replace('Consumer Loan','Vehicle loan or lease',inplace=True)
df['Sub-product'].replace('Vehicle lease','Lease',inplace=True)
df['Sub-product'].replace('Vehicle loan','Loan',inplace=True)

# Credit reporting
df['Product'].replace('Credit reporting','Credit reporting, credit repair services, or other personal consumer reports',inplace=True)

In [13]:
df[df['Product'] =='Debt collection']['Sub-product'].value_counts()

Other debt                   86516
I do not know                59638
Credit card debt             58593
Medical debt                 43972
Payday loan debt             11920
Auto debt                     8355
Mortgage debt                 8023
Private student loan debt     5075
Federal student loan debt     4904
Name: Sub-product, dtype: int64

In [9]:
product=df.pivot_table(['Complaint ID','Date received'],['Product','Sub-product'],aggfunc={'Complaint ID':lambda x: x.value_counts().count(),'Date received':max})
product.reset_index(inplace=True)#.to_csv('product_list.csv')
product[product['Sub-product'].duplicated(keep=False)].sort_values('Sub-product')

,Product,Sub-product,Complaint ID,Date received
20,"Credit reporting, credit repair services, or o...",Conventional home mortgage,1,2018-05-22
54,Mortgage,Conventional home mortgage,41720,2020-04-02
35,Debt collection,Non-federal student loan,2881,2017-04-21
69,Student loan,Non-federal student loan,22283,2017-04-21
38,Debt collection,Payday loan,7563,2017-04-21
65,"Payday loan, title loan, or personal loan",Payday loan,3781,2020-03-30
67,"Payday loan, title loan, or personal loan",Title loan,1128,2020-03-30
73,Vehicle loan or lease,Title loan,562,2019-11-19


Remain duplicate records exist in the classification of CFPB website.


### Merge issues under same category.

Group the issues and sub issues

In [11]:
issuedate=df.pivot_table(['Complaint ID','Date received'],['Issue','Sub-issue'],aggfunc={'Complaint ID':lambda x: x.value_counts().count(),'Date received':max}).reset_index()
issuedate.head()#.pivot_table(['Sub-issue','Date received'],['Sub-issue'],aggfunc={'Sub-issue':lambda x:x.count(),'Date received':max})

,Issue,Sub-issue,Complaint ID,Date received
0,Advertising,Changes in terms from what was offered or adve...,33,2020-03-10
1,Advertising,Confusing or misleading advertising about the ...,107,2020-03-23
2,"Advertising and marketing, including promotion...",Confusing or misleading advertising about the ...,1724,2020-03-24
3,"Advertising and marketing, including promotion...",Didn't receive advertised or promotional terms,2648,2020-03-31
4,Attempts to collect debt not owed,Debt is not yours,33236,2020-04-02


Check duplicate sub issues

In [12]:
issuedate[issuedate['Sub-issue'].duplicated(keep=False)].sort_values('Sub-issue').head()#.to_excel('issuedate.xlsx')

,Issue,Sub-issue,Complaint ID,Date received
35,Credit monitoring or identity theft protection...,Billing dispute for services,1624,2020-04-02
91,Identity theft protection or other monitoring ...,Billing dispute for services,58,2020-03-07
11,Closing an account,Can't close your account,1591,2020-03-25
15,Closing your account,Can't close your account,1374,2020-03-26
12,Closing an account,Company closed your account,3088,2020-03-30


In [14]:
df['Issue'].replace('Problem with overdraft','Problem with an overdraft',inplace=True)
#df['Sub-product'].replace('Check cashing','Check cashing service',inplace=True)

df['Issue'].replace("Problem with a company's investigation into an existing issue","Problem with a credit reporting company's investigation into an existing problem",inplace=True)
df['Issue'].replace("Getting a loan","Getting a loan or lease",inplace=True)
df['Issue'].replace("Identity theft protection or other monitoring services","Credit monitoring or identity theft protection services",inplace=True)
df['Issue'].replace("Closing your account","Closing an account",inplace=True)
df['Issue'].replace("Attempts to collect debt not owed","Cont'd attempts collect debt not owed",inplace=True)
df['Issue'].replace("Problem with an overdraft","Problem with an overdraft",inplace=True)
df['Issue'].replace("Struggling to pay your bill","Struggling to repay your loan",inplace=True)

In [14]:
issuedate.reset_index().to_excel('issuedate.xlsx')

## Exploration of data

### We want to use plots to show the relationship among products and issues. The future steps of visualization will finished in R.

In [123]:
df.head(2)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500


In [16]:
df=df.rename(columns={'Complaint ID':'id'})

In [17]:
import datetime
df['year']=pd.DatetimeIndex(df['Date received']).year
df['month']=pd.DatetimeIndex(df['Date received']).month

In [15]:
df.to_csv('cleancomplaint.csv',index=False)

In [45]:
from google.colab import files
files.download('cleancomplaint.csv' )


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>